In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from torchsummary import summary

In [3]:
#activation fumction ReLU and its derivative
def ReLu_d(x):
    return np.where(x < 0, 0, 1)
def ReLu(x):
    return np.where(x < 0, 0, x)

In [30]:
#Tensor->（multidimensional array）

#d input nodes k hidden layer
K = 10
d = 10
batch_size = 1

X = Variable(torch.rand(batch_size,d),requires_grad=True)
y = torch.tensor([[torch.sum(X**2)/d]])

In [57]:
class NN1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        global z,z_relu
        z = []
        z_relu = []
        super(NN1, self).__init__()
        self.layer = nn.Sequential()
        self.layer.add_module("h_1", nn.Linear(input_size, hidden_size))
        z.append(self.layer(X))
        self.layer.add_module("h_1_relu",nn.ReLU())
        z_relu.append(self.layer(X))
        
        for i in range(K-1):
             index = str(i+2)
             self.layer.add_module("h_"+index, nn.Linear(hidden_size, hidden_size))
             z.append(self.layer(X)) 
             self.layer.add_module("h_"+index+"_relu"+index,nn.ReLU())
             z_relu.append(self.layer(X))
                
        self.layer.add_module("h_last", nn.Linear(hidden_size, num_classes))
        z.append(self.layer(X))
        #self.layer.add_module("h_1ast_relu"+index,nn.ReLU())
        z_relu.append(self.layer(X))

    def forward(self, X):
        y = self.layer(X)
        return y
        

In [58]:
model = NN1(d,2*d+1,1)

In [59]:
y_pred = model(X)
loss = (y_pred - y)**2
loss.item()


0.1346285045146942

In [60]:
y_pred

tensor([[-0.10352]], grad_fn=<AddmmBackward>)

In [61]:
summary(model,(1,10))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 21]             231
              ReLU-2                [-1, 1, 21]               0
            Linear-3                [-1, 1, 21]             462
              ReLU-4                [-1, 1, 21]               0
            Linear-5                [-1, 1, 21]             462
              ReLU-6                [-1, 1, 21]               0
            Linear-7                [-1, 1, 21]             462
              ReLU-8                [-1, 1, 21]               0
            Linear-9                [-1, 1, 21]             462
             ReLU-10                [-1, 1, 21]               0
           Linear-11                [-1, 1, 21]             462
             ReLU-12                [-1, 1, 21]               0
           Linear-13                [-1, 1, 21]             462
             ReLU-14                [-1

In [62]:
loss.backward()

--------------------

In [79]:
#for param in model.named_parameters():
    #print(param)

In [64]:
#save auto w,b
auto_w =[]
auto_b =[]
i = 0

for param in model.parameters():
    if (i%2==0):
        #auto_w.append(param.grad)
        auto_w.append(np.round(np.array(param.grad).astype(np.double),5))
    if (i%2==1):
        #auto_b.append(param.grad)
         auto_b.append(np.round(np.array(param.grad).astype(np.double),5))
    i+=1

-----------------------

In [65]:
#手动grad share the same weight and bias with auto_forward
parameters = list(model.parameters())

In [66]:
#manual forward
def feedforward(X,parameters):
    global z_manual,z_relu_manual
    z_manual=[]
    z_relu_manual=[]
    z_manual.append(torch.from_numpy(np.dot(X.detach().numpy(),(parameters[0].detach().numpy().T))+parameters[1].detach().numpy()))
    z_relu_manual.append(ReLu(z_manual[0]))
    
    for i in range(0,10):
        z_manual.append(torch.from_numpy(np.dot(z_relu_manual[i],(parameters[(i+1)*2].detach().numpy().T))+parameters[(i+1)*2+1].detach().numpy()))
        z_relu_manual.append(ReLu(z_manual[i+1]))
    z_relu_manual[-1]=z_manual[-1]
    return z_relu_manual[-1]
y_pred = feedforward(X,parameters)

In [67]:
diff_z=0
for i in range(len(z)):
    diff_z = diff_z+ np.absolute(sum(sum(z[i].detach().numpy()-z_manual[i].detach().numpy())))
diff_z

0.0

In [68]:
diff_z_relu=0
for i in range(len(z_relu)):
    diff_z_relu = diff_z_relu+ np.absolute(sum(sum(z_relu[i].detach().numpy()-np.array(z_relu_manual[i]))))
diff_z_relu

0.0

In [69]:
#求对z的的倒数
# dL/dv5
d_z = []
d_w = []
torch.set_printoptions(precision=5)
grad_y_pred = 2.0 * (y_pred - y) #[1,1]
grad_z_y = grad_y_pred.detach().numpy()#*ReLu_d(y_pred)#dL/dz5,5)
grad_w_y = grad_z_y*z_relu_manual[-2]#.detach().numpy()
d_z.append(grad_z_y)
d_w.append(grad_w_y)
#from the last hidden layer
for i in range (9):#i=1,2,3,4,5.... p:-4,-6,-8
    grad_z_i = np.dot(d_z[i],ReLu_d(z_manual[-i-2])*parameters[-(i+1)*2].detach().numpy())
    d_z.append(grad_z_i)
    grad_w_i = np.dot(grad_z_i.T,z_relu_manual[-3-i])#.detach().numpy() )#-4,-5,-6
    d_w.append(grad_w_i)
    
#first layer
grad_z_1 = np.dot(d_z[9],ReLu_d(z_manual[-11])*parameters[-20].detach().numpy())
d_z.append(grad_z_1)
grad_w_1 = np.dot(grad_z_1.T,X.detach().numpy())
d_w.append(grad_w_1)


-------------

In [70]:
for i in range(len(d_w)):
    d_w[i]= np.round(np.array(d_w[i]).astype(np.double),5)
    d_z[i]= np.round(np.array(d_z[i]).astype(np.double),5)

In [71]:
d_w.reverse()

In [72]:
d_z.reverse()

In [73]:
#chaeck value difference between autograd and mygrad

In [74]:
diff_w=0
for i in range(len(d_w)):
    diff_w = diff_w+ np.absolute(sum(sum(d_w[i]-auto_w[i])))

In [75]:
diff_w

0.0

In [76]:
diff_b=0
for i in range(len(d_z)):
    diff_b = diff_b+ np.absolute(sum(sum(d_z[i]-auto_b[i])))
diff_b

0.0

In [77]:
with open('torch_autograd.dat', 'w') as f:
    f.write('autograd_w' + '\n')
    i = 1
    for w in auto_w:
        f.write('weight_' +str(i)+ '\n')
        f.write(str(w)+'\n')
        i+=1
    f.write('autograd_b' + '\n')
    i = 1
    for b in auto_b:
        f.write('bias_' +str(i)+ '\n')
        f.write(str(b)+'\r\n')
        i+=1
    

In [78]:
with open('my_autograd.dat', 'w') as f:
    f.write('my_w' + '\n')
    i = 1
    for w in d_w:
        f.write('weight_' +str(i)+ '\n')
        f.write(str(w)+'\n')
        i+=1
    f.write('my_b' + '\n')
    i = 1
    for b in d_z:
        f.write('bias_' +str(i)+ '\n')
        f.write(str(b)+'\r\n')
        i+=1
    